# Segmenting and Clustering Toronto Neighborhoods
# Part 1

In [3]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Set url

In [4]:
website_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Get website page

In [5]:
source = requests.get(website_url).text

Parse using BeautifulSoup

In [6]:
soup = BeautifulSoup(source,'lxml')

Extract the table using "find". Assume only one table on the page.

In [7]:
table = soup.find('table',class_="wikitable sortable")

Extract all of the rows from the table (tags "th" and "td")

In [8]:
mylist = []
for items in table.find_all("tr")[:-1]:
    data = [' '.join(item.text.split()) for item in items.find_all(['th','td'])]
    mylist.append(data)

convert list to dataframe and set column names to first row. Drop first row.

In [9]:
df = pd.DataFrame(mylist)
df.columns = df.iloc[0]
df=df.drop(df.index[0])

Drop "Not Assigned" Boroughs

In [10]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)

Group by Postcode & Borough and concatenate the Neighbourhood data

In [11]:
df_group = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(' ,'.join).reset_index()

Assign the Borough name to the Neighborhood if Neighborhood is "Not assigned"

In [12]:
df_group.Neighbourhood.loc[df_group.Neighbourhood == "Not assigned"] = df_group.Borough

In [13]:
df_group.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park ,Ionview ,Kennedy Park"
7,M1L,Scarborough,"Clairlea ,Golden Mile ,Oakridge"
8,M1M,Scarborough,"Cliffcrest ,Cliffside ,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff ,Cliffside West"


In [14]:
df_group.shape

(103, 3)